In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

In [ ]:
# !pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
spark

In [ ]:
bucket_name = 'ir_3_207472234' 
relative_path = f"indexes/anchor_text"
client = storage.Client()

In [ ]:
paths = "gs://ir_3_207472234/multistream*_preprocessed.parquet"

parquetFile = spark.read.parquet(paths)

doc_ids_rdd = parquetFile.select( "id").rdd

In [ ]:
def doc_id_stats(doc_ids_rdd):
    """
    doc_ids_rdd: RDD of Rows or ints containing document ids

    Returns:
        N       : number of documents
        max_id  : maximum document id
        min_id  : minimum document id
        density : N / (max_id + 1)
    """
    # convert Row -> int if needed
    ids = doc_ids_rdd.map(lambda r: int(r[0]) if not isinstance(r, int) else r)

    N = ids.count()
    max_id = ids.max()
    min_id = ids.min()

    density = N / (max_id + 1) if max_id >= 0 else 0.0

    return {
        "N": N,
        "max_id": max_id,
        "min_id": min_id,
        "density": density
    }


In [ ]:
stats = doc_id_stats(doc_ids_rdd)
print(stats)